In [46]:
from csv import reader
import numpy as np
from unidecode import unidecode
import pandas as pd
import statsmodels.api as sm
import scipy as ss
from sklearn import preprocessing

In [ ]:
from limpeza import main as limpa
from cria_legislatura import main as legislatura
from build import main as build
from gml_para_rede import main as rede
from conta_frentes import main as frentes

limpa()
legislatura()
build()
rede()
frentes()

#### Nesse teste de hipótese estariamos avaliando qual a influência das mulheres nas votações, será que os homens possuem além de maior representativade maior facilidade para aprovação de propostas?

In [2]:
deputados = pd.read_csv('ArquivosLimpos/deputados-2015-2018.csv').dropna()

In [3]:
deputados.dtypes

deputado_id                 int64
nome                       object
idLegislaturaInicial        int64
idLegislaturaFinal          int64
siglaSexo                  object
betweenness_centrality    float64
degree_centrality         float64
eigenvecto_centrality     float64
Frentes                     int64
dtype: object

In [4]:
values = deputados['siglaSexo'].unique()

for value in values[:-1]:
    deputados[value] = [1 if v == value else 0 for v in deputados['siglaSexo']]

In [5]:
deputados['F'] = deputados['siglaSexo'].apply(lambda value: 1 if value == "F" else 0)

In [6]:
deputados

,deputado_id,nome,idLegislaturaInicial,idLegislaturaFinal,siglaSexo,betweenness_centrality,degree_centrality,eigenvecto_centrality,Frentes,M,F
0,4930,SILVIO COSTA,53,55,M,0.0,0.090610,4.965024e-11,63,1,0
1,4931,IZALCI LUCAS,53,55,M,0.0,0.090610,4.965024e-11,195,1,0
2,62881,Danilo Forte,54,56,M,0.0,0.087315,4.965024e-11,227,1,0
3,64960,MARIA HELENA,52,55,F,0.0,0.080725,4.965024e-11,99,0,1
4,66179,Norma Ayub,55,56,F,0.0,0.046129,4.965024e-11,8,0,1
...,...,...,...,...,...,...,...,...,...,...,...
603,200153,PROF. GEDEAO AMORIM,55,55,M,0.0,0.014827,4.965024e-11,6,1,0
604,202063,OSVALDO MAFRA,55,55,M,0.0,0.011532,4.965024e-11,2,1,0
605,202915,EURICELIA CARDOSO,55,55,F,0.0,0.006590,4.965024e-11,0,0,1
606,204011,PROFESSOR PACCO,55,55,M,0.0,0.006590,4.965024e-11,0,1,0


#### Influênca sexo na influência de um deputado

In [7]:
model = sm.Logit(deputados['eigenvecto_centrality'], deputados[values[:-1]])
result = model.fit()
result.summary()

Optimization terminated successfully.
         Current function value: 0.085200
         Iterations 8


c:\users\duda\appdata\local\programs\python\python37-32\lib\site-packages\statsmodels\discrete\discrete_model.py:3489: RuntimeWarning: divide by zero encountered in double_scalars
  return 1 - self.llf/self.llnull


<class 'statsmodels.iolib.summary.Summary'>
"""
                             Logit Regression Results                            
=================================================================================
Dep. Variable:     eigenvecto_centrality   No. Observations:                  608
Model:                             Logit   Df Residuals:                      607
Method:                              MLE   Df Model:                            0
Date:                   Sun, 22 Nov 2020   Pseudo R-squ.:                     inf
Time:                           14:47:56   Log-Likelihood:                -51.801
converged:                          True   LL-Null:                        0.0000
Covariance Type:               nonrobust   LLR p-value:                       nan
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
M             -4.3015      0.373    -11.543      0.000      -5.032      -3.571
==============================================================================
"""

#### Influência eficiência na inflência de um deputado

In [8]:
model = sm.OLS(deputados['eigenvecto_centrality'], deputados[['Frentes']])
result = model.fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                  OLS Regression Results                                  
==========================================================================================
Dep. Variable:     eigenvecto_centrality   R-squared (uncentered):                   0.096
Model:                               OLS   Adj. R-squared (uncentered):              0.094
Method:                    Least Squares   F-statistic:                              64.29
Date:                   Sun, 22 Nov 2020   Prob (F-statistic):                    5.54e-15
Time:                           14:47:57   Log-Likelihood:                          1116.6
No. Observations:                    608   AIC:                                     -2231.
Df Residuals:                        607   BIC:                                     -2227.
Df Model:                              1                                                  
Covariance Type:               nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Frentes      8.58e-05   1.07e-05      8.018      0.000    6.48e-05       0.000
==============================================================================
Omnibus:                      337.043   Durbin-Watson:                   1.927
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1597.187
Skew:                           2.651   Prob(JB):                         0.00
Kurtosis:                       8.911   Cond. No.                         1.00
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [9]:
cor = np.corrcoef(deputados['eigenvecto_centrality'],deputados['degree_centrality'])

In [10]:
cor

array([[1.        , 0.98913504],
       [0.98913504, 1.        ]])

In [11]:
H = deputados.query('siglaSexo == "M"')['eigenvecto_centrality']
M = deputados.query('siglaSexo == "F"')['eigenvecto_centrality']

In [12]:
deputados.groupby('siglaSexo').describe()

deputado_id                                                  \
                count           mean           std      min       25%   
siglaSexo                                                               
F                62.0  147608.241935  46170.719957  64960.0  98839.25   
M               546.0  139009.498168  44734.778919   4930.0  74707.00   

                                         idLegislaturaInicial             ...  \
                50%        75%       max                count       mean  ...   
siglaSexo                                                                 ...   
F          175225.5  178945.75  202915.0                 62.0  53.919355  ...   
M          160535.5  178881.75  204760.0                546.0  53.161172  ...   

             M           F                                     
           75%  max  count mean  std  min  25%  50%  75%  max  
siglaSexo                                                      
F          0.0  0.0   62.0  1.0  0.0  1.0  1.0  1.0  1.0  1.0  
M          1.0  1.0  546.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  

[2 rows x 72 columns]

In [13]:
res = ss.stats.ttest_ind(H, M, equal_var=True)

In [14]:
res

Ttest_indResult(statistic=-0.7644941603797848, pvalue=0.44487039484630153)

In [15]:
ttest,pval = ss.stats.ttest_ind(H, M)
print("p-value",pval)

p-value 0.44487039484630153


In [16]:
dgbd= deputados.groupby(['nome']).sum()

In [25]:
dgbd[dgbd.eigenvecto_centrality>0.15]

,deputado_id,idLegislaturaInicial,idLegislaturaFinal,betweenness_centrality,degree_centrality,eigenvecto_centrality,Frentes,M,F
nome,,,,,,,,,
ANDRE MOURA,160543,54,55,0.001965,0.945634,0.167586,213,1,0
ARNALDO FARIA DE SA,73434,48,55,0.008059,1.011532,0.169037,121,1,0
Afonso Florence,160508,54,56,0.002209,0.965404,0.162351,155,1,0
Arthur Oliveira Maia,160600,54,56,0.004409,1.018122,0.169189,129,1,0
CABO SABINO,178865,55,55,0.004242,0.897858,0.157366,139,1,0
JORGINHO MELLO,160509,54,55,0.003670,0.889621,0.154529,215,1,0
Jandira Feghali,74848,49,56,0.002878,0.945634,0.158993,54,0,1
Joao Campos,74366,52,56,0.004437,0.942339,0.162756,173,1,0
LEONARDO PICCIANI,74254,52,55,0.000695,0.795717,0.154953,111,1,0


In [26]:
dgbd[dgbd.degree_centrality>0.877]

,deputado_id,idLegislaturaInicial,idLegislaturaFinal,betweenness_centrality,degree_centrality,eigenvecto_centrality,Frentes,M,F
nome,,,,,,,,,
ALBERTO FRAGA,73579,51,55,0.002340,0.878089,0.140611,191,1,0
ANDRE MOURA,160543,54,55,0.001965,0.945634,0.167586,213,1,0
ARNALDO FARIA DE SA,73434,48,55,0.008059,1.011532,0.169037,121,1,0
Afonso Florence,160508,54,56,0.002209,0.965404,0.162351,155,1,0
Alice Portugal,74057,52,56,0.001759,0.902801,0.143808,194,0,1
Arthur Oliveira Maia,160600,54,56,0.004409,1.018122,0.169189,129,1,0
CABO SABINO,178865,55,55,0.004242,0.897858,0.157366,139,1,0
JORGINHO MELLO,160509,54,55,0.003670,0.889621,0.154529,215,1,0
Jandira Feghali,74848,49,56,0.002878,0.945634,0.158993,54,0,1


In [149]:
H = dgbd.query('F == 0')['eigenvecto_centrality']
M = dgbd.query('F == 1')['eigenvecto_centrality']
H

nome
ABEL MESQUITA JR.       1.120451e-01
ADAIL CARNEIRO          4.965024e-11
ADALBERTO CAVALCANTI    4.965024e-11
ADELMO CARNEIRO LEAO    4.965024e-11
ADELSON BARRETO         4.965024e-11
                            ...     
ZECA CAVALCANTI         4.965024e-11
ZECA DO PT              4.965024e-11
Ze Carlos               4.965024e-11
Ze Silva                4.965024e-11
Zeca Dirceu             4.965024e-11
Name: eigenvecto_centrality, Length: 546, dtype: float64

In [164]:
d=dgbd.sort_values(by='eigenvecto_centrality', ascending=False).head(100)

d

,deputado_id,idLegislaturaInicial,idLegislaturaFinal,betweenness_centrality,degree_centrality,eigenvecto_centrality,Frentes,M,F
nome,,,,,,,,,
Arthur Oliveira Maia,160600,54,56,0.004409,1.018122,0.169189,129,1,0
ARNALDO FARIA DE SA,73434,48,55,0.008059,1.011532,0.169037,121,1,0
ANDRE MOURA,160543,54,55,0.001965,0.945634,0.167586,213,1,0
WALTER IHOSHI,141560,53,55,0.003187,0.971993,0.166546,154,1,0
Joao Campos,74366,52,56,0.004437,0.942339,0.162756,173,1,0
...,...,...,...,...,...,...,...,...,...
Paulo Foletto,160517,54,56,0.000177,0.128501,0.006762,246,1,0
Celso Russomanno,73441,50,56,0.000083,0.116969,0.006237,139,1,0
Afonso Hamm,136811,52,56,0.000138,0.141680,0.006030,180,1,0


In [179]:
H.mean()
d=dgbd.sort_values(by='eigenvecto_centrality', ascending=False).head(200)
H = d.query('F == 0')['eigenvecto_centrality']
M = dgbd.query('F == 1')['eigenvecto_centrality']
M
H

nome
Arthur Oliveira Maia    1.691888e-01
ARNALDO FARIA DE SA     1.690374e-01
ANDRE MOURA             1.675864e-01
WALTER IHOSHI           1.665461e-01
Joao Campos             1.627560e-01
                            ...     
MAURO MARIANI           4.965024e-11
MAURO PEREIRA           4.965024e-11
MAX FILHO               4.965024e-11
Nelson Pellegrino       4.965024e-11
NILTON CAPIXABA         4.965024e-11
Name: eigenvecto_centrality, Length: 177, dtype: float64

In [180]:
# deputados[deputados.F== 1].eigenvecto_centrality


In [181]:
# H0 --> data é normal 
ss.stats.shapiro(H)

ShapiroResult(statistic=0.7212174534797668, pvalue=6.074611263030992e-17)

In [182]:

ss.stats.shapiro(M)

ShapiroResult(statistic=0.46352270245552063, pvalue=1.062875780227443e-13)

In [183]:
# teste = dgbd.iloc[:,[5, 8]]
# homem= teste[teste.F ==0]
# mulher= teste[teste.F ==1]
# mulher

In [184]:
# xh = np.array(H)
# xm = np.array(M)


In [185]:
# MN= preprocessing.normalize(xm)
# HN= preprocessing.normalize(xh)

In [186]:
# ss.stats.levene(H, M)

In [187]:
res = ss.stats.ttest_ind(M,H, equal_var=True)
res

Ttest_indResult(statistic=-3.029028883101098, pvalue=0.0027250848334898488)

In [188]:
# ttest,pval = ss.stats.ttest_ind(H, M)
# print("p-value",pval)
# print("t-value",ttest)

In [189]:
### Ou seja mean não é igual e mulher tem menos prestigio que homem.

In [193]:
dgbd.sort_values(by='eigenvecto_centrality', ascending=False).head(15)

,deputado_id,idLegislaturaInicial,idLegislaturaFinal,betweenness_centrality,degree_centrality,eigenvecto_centrality,Frentes,M,F
nome,,,,,,,,,
Arthur Oliveira Maia,160600,54,56,0.004409,1.018122,0.169189,129,1,0
ARNALDO FARIA DE SA,73434,48,55,0.008059,1.011532,0.169037,121,1,0
ANDRE MOURA,160543,54,55,0.001965,0.945634,0.167586,213,1,0
WALTER IHOSHI,141560,53,55,0.003187,0.971993,0.166546,154,1,0
Joao Campos,74366,52,56,0.004437,0.942339,0.162756,173,1,0
Afonso Florence,160508,54,56,0.002209,0.965404,0.162351,155,1,0
Jandira Feghali,74848,49,56,0.002878,0.945634,0.158993,54,0,1
CABO SABINO,178865,55,55,0.004242,0.897858,0.157366,139,1,0
LEONARDO PICCIANI,74254,52,55,0.000695,0.795717,0.154953,111,1,0


In [195]:
# Amostra completa homens
H = dgbd.query('F == 0')['eigenvecto_centrality']
M = dgbd.query('F == 1')['eigenvecto_centrality']

res = ss.stats.ttest_ind(M,H, equal_var=True)
res

Ttest_indResult(statistic=0.7644941603797838, pvalue=0.4448703948463021)

In [196]:
H

nome
ABEL MESQUITA JR.       1.120451e-01
ADAIL CARNEIRO          4.965024e-11
ADALBERTO CAVALCANTI    4.965024e-11
ADELMO CARNEIRO LEAO    4.965024e-11
ADELSON BARRETO         4.965024e-11
                            ...     
ZECA CAVALCANTI         4.965024e-11
ZECA DO PT              4.965024e-11
Ze Carlos               4.965024e-11
Ze Silva                4.965024e-11
Zeca Dirceu             4.965024e-11
Name: eigenvecto_centrality, Length: 546, dtype: float64